In [1]:
import numpy as np
import pandas as pd
import yfinance
import statsmodels.formula.api as smf
import statsmodels.api as smi
from table_making import summary_col
import matplotlib.pyplot as plt
import re
import os

In [2]:
if not os.path.exists("tables/iv"):
    os.makedirs("tables/iv")

In [3]:
def latex_format(sm):
    ltx = sm.as_latex()
    ltx = re.sub(r" +", " ", ltx)
    ltx = ltx.replace(" & iv\\_onem & delta\\_iv \\\\\n\\hline", """ & $x_t[1/12, 1]$ & $|x_t[1/12, 1] - x_{t-1}[1/12, 1]|$ \\\\
 & (1) & (2) \\\\
\cmidrule(lr){2-2} \cmidrule(lr){3-3}""")
    ltx = ltx.replace("iv\_atm\_std", "$\sigma(\\text{iv}_{\\text{ATM}})$")
    ltx = ltx.replace("iv\_atm\_mean", "$\mu(\\text{iv}_{\\text{ATM}})$")
    ltx = ltx.replace("R-squared Adj.", "\\hline\n$R^2$")
    ltx = ltx.replace("\\$N\\$", "$N$")
    ltx = ltx.replace("""lll""", """lcc""")
    ltx = ltx.replace("cc}\n\\hline", "cc}\n\\hline\\hline")
    ltx = ltx.replace("\\begin{table}\n\\caption{}\n\\label{}\n\\begin{center}\n", "")
    ltx = ltx.replace("\\end{center}\n\\end{table}", "")
    return ltx

# Regression - no ex feats

## std

In [4]:
spx_df = pd.read_parquet("data/spx_with_vix.parquet")
spx_df["iv_onem"] = spx_df["ttm_one_month_moneyness_pt_one"]
spx_df["delta_iv"] = np.abs(spx_df["iv_onem"] - spx_df["iv_onem"].shift(1))
simulated_surfaces = np.load("test_spx/2023_08_27_simple/model_0_gen3.npy")
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_std = np.std(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, ["date", "iv_onem", "delta_iv"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_std"] = simulated_atm_std

models = []
for y in ["iv_onem", "delta_iv"]:
    model = smf.ols(f"{y}~1+iv_atm_std", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["iv_onem", "delta_iv"], stars=True,
                 regressor_order=["iv_atm_std", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/iv/reg_overall_no_ex_std.tex", "w") as f:
    f.write(latex_format(sm))

## mean

In [5]:
spx_df = pd.read_parquet("data/spx_with_vix.parquet")
spx_df["iv_onem"] = spx_df["ttm_one_month_moneyness_pt_one"]
spx_df["delta_iv"] = np.abs(spx_df["iv_onem"] - spx_df["iv_onem"].shift(1))
simulated_surfaces = np.load("test_spx/2023_08_27_simple/model_0_gen3.npy")
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_mean = np.mean(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, ["date", "iv_onem", "delta_iv"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_mean"] = simulated_atm_mean

models = []
for y in ["iv_onem", "delta_iv"]:
    model = smf.ols(f"{y}~1+iv_atm_mean", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["iv_onem", "delta_iv"], stars=True,
                 regressor_order=["iv_atm_mean", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/iv/reg_overall_no_ex_mean.tex", "w") as f:
    f.write(latex_format(sm))

## Regression - no loss

## std

In [6]:
spx_df = pd.read_parquet("data/spx_with_vix.parquet")
spx_df["iv_onem"] = spx_df["ttm_one_month_moneyness_pt_one"]
spx_df["delta_iv"] = np.abs(spx_df["iv_onem"] - spx_df["iv_onem"].shift(1))
simulated_surfaces = np.load("test_spx/2023_08_27/model_0_gen3.npy")
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_std = np.std(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, ["date", "iv_onem", "delta_iv"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_std"] = simulated_atm_std

models = []
for y in ["iv_onem", "delta_iv"]:
    model = smf.ols(f"{y}~1+iv_atm_std", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["iv_onem", "delta_iv"], stars=True,
                 regressor_order=["iv_atm_std", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/iv/reg_overall_no_loss_std.tex", "w") as f:
    f.write(latex_format(sm))

## mean

In [7]:
spx_df = pd.read_parquet("data/spx_with_vix.parquet")
spx_df["iv_onem"] = spx_df["ttm_one_month_moneyness_pt_one"]
spx_df["delta_iv"] = np.abs(spx_df["iv_onem"] - spx_df["iv_onem"].shift(1))
simulated_surfaces = np.load("test_spx/2023_08_27/model_0_gen3.npy")
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_mean = np.mean(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, ["date", "iv_onem", "delta_iv"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_mean"] = simulated_atm_mean

models = []
for y in ["iv_onem", "delta_iv"]:
    model = smf.ols(f"{y}~1+iv_atm_mean", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["iv_onem", "delta_iv"], stars=True,
                 regressor_order=["iv_atm_mean", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/iv/reg_overall_no_loss_mean.tex", "w") as f:
    f.write(latex_format(sm))

# Regression - loss

## std

In [8]:
spx_df = pd.read_parquet("data/spx_with_vix.parquet")
spx_df["iv_onem"] = spx_df["ttm_one_month_moneyness_pt_one"]
spx_df["delta_iv"] = np.abs(spx_df["iv_onem"] - spx_df["iv_onem"].shift(1))
simulated_surfaces = np.load("test_spx/2023_08_27/model_1_gen3.npz")["surfaces"]
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_std = np.std(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, ["date", "iv_onem", "delta_iv"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_std"] = simulated_atm_std

models = []
for y in ["iv_onem", "delta_iv"]:
    model = smf.ols(f"{y}~1+iv_atm_std", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["iv_onem", "delta_iv"], stars=True,
                 regressor_order=["iv_atm_std", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/iv/reg_overall_loss_std.tex", "w") as f:
    f.write(latex_format(sm))

## mean

In [9]:
spx_df = pd.read_parquet("data/spx_with_vix.parquet")
spx_df["iv_onem"] = spx_df["ttm_one_month_moneyness_pt_one"]
spx_df["delta_iv"] = np.abs(spx_df["iv_onem"] - spx_df["iv_onem"].shift(1))
simulated_surfaces = np.load("test_spx/2023_08_27/model_1_gen3.npz")["surfaces"]
simulated_atm = simulated_surfaces[:, :, 3, 2]
simulated_atm_mean = np.mean(simulated_atm, axis=1)

regression_df = spx_df.loc[3:5303-1, ["date", "iv_onem", "delta_iv"]]
regression_df1 = regression_df.copy()
regression_df1["iv_atm_mean"] = simulated_atm_mean

models = []
for y in ["iv_onem", "delta_iv"]:
    model = smf.ols(f"{y}~1+iv_atm_mean", data=regression_df1).fit(cov_type="HC3")
    models.append(model)

sm = summary_col(models, model_names=["iv_onem", "delta_iv"], stars=True,
                 regressor_order=["iv_atm_mean", "Intercept"],
                 info_dict={'$N$':lambda x: "{0:d}".format(int(x.nobs))})
with open("tables/iv/reg_overall_loss_mean.tex", "w") as f:
    f.write(latex_format(sm))